In [4]:
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import re

# ==== ПАРАМЕТРЫ ====
ROOT = Path("../../preds_x2")   # папка с результатами
OUT_PATH = Path("../../preds_test3_crops_montage.png")

# 1) ВАРИАНТ A: задать box относительно центра (ширина/высота в пикселях)
BOX_SIZE = (100, 100)      # (w, h) «небольшой» box
BOX_OFFSET = (0, 0)      # смещение от центра (dx, dy). 0,0 = ровно центр

# 2) ВАРИАНТ B: точные координаты (x0, y0, x1, y1)
USE_ABSOLUTE_BOX = False
ABS_BOX = (50, 50, 114, 114)

# Масштаб кропов и интерполяция
UPSCALE = 4
RESAMPLE = Image.NEAREST   # Image.NEAREST / BICUBIC / BILINEAR / LANCZOS

# «Оригинал» для отображения рамки: 'hr' | 'lr' | 'pred'
ORIG_KIND = "hr"
# Предпросмотр «оригинала»: приводим по высоте к высоте тайла кропа
ORIG_PREVIEW_MATCH_CROP_HEIGHT = True
# Толщина красной рамки на «оригинале» (после ресайза предпросмотра)
RECT_WIDTH = 3

# Визуальные параметры
PADDING = 8         # поля внутри тайла
GUTTER = 12         # расстояния между ячейками
LABEL_HEIGHT = 22   # место под подпись
FONT_SIZE = 35  # размер шрифта (подбери под себя: 24, 28, 32 и т.п.)
try:
    # на Windows часто достаточно указать просто "arial.ttf"
    FONT = ImageFont.truetype("arial.ttf", FONT_SIZE)
except OSError:
    # если шрифт не найдён, fallback на стандартный
    FONT = ImageFont.load_default()

# высота области под подпись: чуть больше, чем размер шрифта
LABEL_HEIGHT = FONT_SIZE + 10

# ====== ВСПОМОГАТЕЛЬНОЕ ======
def find_triplets(folder: Path):
    """
    Ищет тройки *_lr.*, *_hr.*, *_pred.*.
    Возвращает [{'id': base, 'lr': Path, 'hr': Path, 'pred': Path}, ...]
    """
    exts = {".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp", ".webp"}
    files = [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in exts]

    rx = re.compile(r"^(?P<base>.*?)[_-]?(?P<kind>lr|hr|pred)$", re.IGNORECASE)
    buckets = {}
    for f in files:
        m = rx.match(f.stem)
        if not m:
            continue
        base = m.group("base")
        kind = m.group("kind").lower()
        d = buckets.setdefault(base, {})
        d[kind] = f

    triplets = []
    for base, d in sorted(buckets.items()):
        if all(k in d for k in ("lr", "hr", "pred")):
            triplets.append({"id": base, "lr": d["lr"], "hr": d["hr"], "pred": d["pred"]})
    return triplets

def compute_center_box(w, h, box_w, box_h, dx=0, dy=0):
    cx, cy = w // 2 + dx, h // 2 + dy
    x0 = max(0, cx - box_w // 2)
    y0 = max(0, cy - box_h // 2)
    x1 = min(w, x0 + box_w)
    y1 = min(h, y0 + box_h)
    x0 = max(0, x1 - box_w)
    y0 = max(0, y1 - box_h)
    return (x0, y0, x1, y1)

def unify_sizes(img_lr, img_hr, img_pred):
    """
    Выравниваем LR и PRED под размер HR (целевой), чтобы box совпадал.
    Возвращаем (lr', hr, pred', target_size)
    """
    target_size = img_hr.size
    if img_lr.size != target_size:
        img_lr = img_lr.resize(target_size, Image.BICUBIC)
    if img_pred.size != target_size:
        img_pred = img_pred.resize(target_size, Image.BICUBIC)
    return img_lr, img_hr, img_pred, target_size

def crop_same_box(img_lr, img_hr, img_pred):
    img_lr_, img_hr_, img_pred_, (W, H) = unify_sizes(img_lr, img_hr, img_pred)

    if USE_ABSOLUTE_BOX:
        box = ABS_BOX
    else:
        box = compute_center_box(W, H, BOX_SIZE[0], BOX_SIZE[1], BOX_OFFSET[0], BOX_OFFSET[1])

    return img_lr_.crop(box), img_hr_.crop(box), img_pred_.crop(box), box, (W, H)

def _text_size(draw, text, font):
    # Pillow 10+: textbbox; старые Pillow: textsize
    try:
        bbox = draw.textbbox((0, 0), text, font=font)
        return bbox[2] - bbox[0], bbox[3] - bbox[1]
    except AttributeError:
        return draw.textsize(text, font=font)

def make_tile(crop_img, label):
    # увеличиваем кроп
    up_w, up_h = crop_img.size[0] * UPSCALE, crop_img.size[1] * UPSCALE
    up = crop_img.resize((up_w, up_h), RESAMPLE)

    # тайл с подложкой и подписью
    tile_w = up_w + PADDING * 2
    tile_h = up_h + PADDING * 2 + LABEL_HEIGHT
    tile = Image.new("RGB", (tile_w, tile_h), (20, 20, 20))
    tile.paste(up, (PADDING, PADDING))

    draw = ImageDraw.Draw(tile)
    text = str(label).upper()
    tw, th = _text_size(draw, text, FONT)
    tx = (tile_w - tw) // 2
    ty = up_h + PADDING + (LABEL_HEIGHT - th) // 2
    draw.text((tx, ty), text, fill=(240, 240, 240), font=FONT)
    return tile

def make_orig_tile(orig_img, box, label_for_kind, crop_tile_height):
    """
    Делает тайл из «оригинального» изображения с красной рамкой box.
    Ресайзит предпросмотр «оригинала» по высоте к высоте тайла кропа (если включено).
    """
    # приводим к RGB для рисования цветной рамки
    if orig_img.mode != "RGB":
        base = orig_img.convert("RGB")
    else:
        base = orig_img.copy()

    W, H = base.size
    # считаем размер тайла-кропа для согласования высоты
    # tile_height = up_h + 2*PADDING + LABEL_HEIGHT
    # up_h = crop_h * UPSCALE
    crop_h = box[3] - box[1]
    up_h = crop_h * UPSCALE
    target_tile_h = up_h + PADDING * 2 + LABEL_HEIGHT

    # масштаб предпросмотра «оригинала»
    if ORIG_PREVIEW_MATCH_CROP_HEIGHT:
        scale = (target_tile_h - PADDING * 2 - LABEL_HEIGHT) / float(H)
        preview_w = max(1, int(W * scale))
        preview_h = max(1, int(H * scale))
        preview = base.resize((preview_w, preview_h), Image.BICUBIC)
        # масштабируем координаты рамки
        sx, sy = scale, scale
        x0, y0, x1, y1 = box
        rect = (int(x0 * sx), int(y0 * sy), int(x1 * sx), int(y1 * sy))
    else:
        preview = base
        rect = box

    # собираем тайл
    tile_w = preview.width + PADDING * 2
    tile_h = preview.height + PADDING * 2 + LABEL_HEIGHT
    tile = Image.new("RGB", (tile_w, tile_h), (20, 20, 20))
    tile.paste(preview, (PADDING, PADDING))

    # рисуем рамку
    draw = ImageDraw.Draw(tile)
    rx0, ry0, rx1, ry1 = rect
    # смещение внутри тайла (из-за PADDING)
    rx0 += PADDING
    rx1 += PADDING
    ry0 += PADDING
    ry1 += PADDING
    # рисуем несколько прямоугольников для имитации толщины
    for k in range(RECT_WIDTH):
        draw.rectangle((rx0 - k, ry0 - k, rx1 + k, ry1 + k), outline=(220, 40, 40))

    # подпись
    text = f"ORIG ({label_for_kind.upper()})"
    tw, th = _text_size(draw, text, FONT)
    tx = (tile_w - tw) // 2
    ty = preview.height + PADDING + (LABEL_HEIGHT - th) // 2
    draw.text((tx, ty), text, fill=(240, 240, 240), font=FONT)
    return tile

def hstack(tiles):
    w = sum(t.width for t in tiles) + GUTTER * (len(tiles) - 1)
    h = max(t.height for t in tiles)
    out = Image.new("RGB", (w, h), (10, 10, 10))
    x = 0
    for i, t in enumerate(tiles):
        out.paste(t, (x, 0))
        x += t.width + (GUTTER if i < len(tiles) - 1 else 0)
    return out

def vstack(rows):
    w = max(r.width for r in rows)
    h = sum(r.height for r in rows) + GUTTER * (len(rows) - 1)
    out = Image.new("RGB", (w, h), (5, 5, 5))
    y = 0
    for i, r in enumerate(rows):
        out.paste(r, ((w - r.width) // 2, y))
        y += r.height + (GUTTER if i < len(rows) - 1 else 0)
    return out

# ====== ОСНОВНОЙ ХОД ======
triplets = find_triplets(ROOT)
if not triplets:
    raise SystemExit(f"Не нашёл троек в {ROOT}. Проверьте имена *_lr/_hr/_pred.")

rows = []
for grp in triplets:
    # загружаем как L — кропы будут монохромные; для «оригинала» конвертируем в RGB позже
    img_lr = Image.open(grp["lr"]).convert("L")
    img_hr = Image.open(grp["hr"]).convert("L")
    img_pred = Image.open(grp["pred"]).convert("L")

    lr_crop, hr_crop, pred_crop, box, target_size = crop_same_box(img_lr, img_hr, img_pred)

    # делаем тайлы кропов
    lr_tile = make_tile(lr_crop, "lr")
    hr_tile = make_tile(hr_crop, "hr")
    pred_tile = make_tile(pred_crop, "pred")

    # выбираем «оригинал» для рамки
    if ORIG_KIND.lower() == "hr":
        orig_src = Image.open(grp["hr"])  # без .convert("L") — сохраним исходные тона
        label = "hr"
    elif ORIG_KIND.lower() == "lr":
        orig_src = Image.open(grp["lr"])
        label = "lr"
    else:
        orig_src = Image.open(grp["pred"])
        label = "pred"

    # выравниваем «оригинал» под target_size (как и кропы)
    if orig_src.size != target_size:
        orig_src = orig_src.resize(target_size, Image.BICUBIC)

    # тайл с рамкой
    # высота тайла кропа (чтобы согласовать предпросмотр «оригинала»)
    crop_tile_height = lr_tile.height  # все кроп-тайлы одинаковой высоты
    orig_tile = make_orig_tile(orig_src, box, label_for_kind=label, crop_tile_height=crop_tile_height)

    row = hstack([lr_tile, hr_tile, pred_tile, orig_tile])
    rows.append(row)

montage = vstack(rows)
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
montage.save(OUT_PATH)
print(f"Готово: {OUT_PATH.resolve()}")
print("Всего троек:", len(triplets))

Готово: C:\Users\Вячеслав\Documents\superresolution\preds_test3_crops_montage.png
Всего троек: 16


In [5]:
from pathlib import Path
from PIL import Image, ImageChops
import numpy as np
import matplotlib.pyplot as plt

# Папка для отчётных картинок
OUT_DIR = Path("../../sr_report_x2_msresunet")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def prepare_triplet(grp):
    """Загружаем LR / HR / SR, приводим к одному размеру (HR)."""
    img_lr = Image.open(grp["lr"]).convert("L")
    img_hr = Image.open(grp["hr"]).convert("L")
    img_sr = Image.open(grp["pred"]).convert("L")
    img_lr_u, img_hr_u, img_sr_u, target = unify_sizes(img_lr, img_hr, img_sr)
    return img_lr_u, img_hr_u, img_sr_u

def make_tile2(img, label, upscale=1):
    """Тайл с подписью (аналог make_tile, но с параметром upscale)."""
    up_w, up_h = img.size[0] * upscale, img.size[1] * upscale
    up = img.resize((up_w, up_h), RESAMPLE)

    tile_w = up_w + PADDING * 2
    tile_h = up_h + PADDING * 2 + LABEL_HEIGHT
    tile = Image.new("RGB", (tile_w, tile_h), (20, 20, 20))
    tile.paste(up, (PADDING, PADDING))

    from PIL import ImageDraw
    draw = ImageDraw.Draw(tile)
    text = str(label)
    tw, th = _text_size(draw, text, FONT)
    tx = (tile_w - tw) // 2
    ty = up_h + PADDING + (LABEL_HEIGHT - th) // 2
    draw.text((tx, ty), text, fill=(240, 240, 240), font=FONT)
    return tile

def make_diff_img(a, b, amplify=1.0):
    """Абсолютная разность |a-b| с возможным усилением контраста."""
    diff = ImageChops.difference(a, b)  # |a - b|
    if amplify != 1.0:
        arr = np.asarray(diff, dtype=np.float32) * float(amplify)
        arr = np.clip(arr, 0, 255).astype("uint8")
        diff = Image.fromarray(arr, mode="L")
    return diff

def otsu_threshold(arr_uint8):
    """Простейшая реализация порога Отсу для uint8-изображения."""
    hist, _ = np.histogram(arr_uint8.ravel(), bins=256, range=(0, 255))
    total = arr_uint8.size
    sum_total = np.dot(np.arange(256), hist)

    sum_b = 0.0
    w_b = 0
    max_var = 0.0
    thresh = 0

    for t in range(256):
        w_b += hist[t]
        if w_b == 0:
            continue
        w_f = total - w_b
        if w_f == 0:
            break
        sum_b += t * hist[t]
        m_b = sum_b / w_b
        m_f = (sum_total - sum_b) / w_f
        var_between = w_b * w_f * (m_b - m_f) ** 2
        if var_between > max_var:
            max_var = var_between
            thresh = t
    return thresh

def binarize_from_hr_threshold(img_hr, img_lr, img_sr):
    """Бинаризация HR/LR/SR с единым порогом, рассчитанным по HR."""
    hr_np = np.asarray(img_hr, dtype=np.uint8)
    lr_np = np.asarray(img_lr, dtype=np.uint8)
    sr_np = np.asarray(img_sr, dtype=np.uint8)

    thr = otsu_threshold(hr_np)

    def _bin(a):
        m = (a > thr).astype("uint8") * 255
        return Image.fromarray(m, mode="L")

    return _bin(hr_np), _bin(lr_np), _bin(sr_np), thr

def xor_mask(a, b):
    """XOR двух бинарных масок (0/255) -> 0/255."""
    a_np = np.asarray(a, dtype=np.uint8) > 0
    b_np = np.asarray(b, dtype=np.uint8) > 0
    xor_np = (a_np ^ b_np).astype("uint8") * 255
    return Image.fromarray(xor_np, mode="L")

# ========= 1. Полноразмерное: HR/LR/SR и разности =========
triplets = find_triplets(ROOT)
if not triplets:
    raise SystemExit("Нет троек для отчёта")
grp = triplets[0]  # можно поменять индекс или выбрать по id

img_lr, img_hr, img_sr = prepare_triplet(grp)

diff_hr_lr = make_diff_img(img_hr, img_lr, amplify=4.0)
diff_hr_sr = make_diff_img(img_hr, img_sr, amplify=4.0)
diff_sr_lr = make_diff_img(img_sr, img_lr, amplify=4.0)

row1 = hstack([
    make_tile2(img_hr, "HR", upscale=1),
    make_tile2(img_lr, "LR↑x2", upscale=1),
    make_tile2(diff_hr_lr, "|HR-LR|", upscale=1),
])
row2 = hstack([
    make_tile2(img_hr, "HR", upscale=1),
    make_tile2(img_sr, "SR", upscale=1),
    make_tile2(diff_hr_sr, "|HR-SR|", upscale=1),
])
row3 = hstack([
    make_tile2(diff_sr_lr, "|SR-LR|", upscale=1),
])

fig1 = vstack([row1, row2, row3])
fig1_path = OUT_DIR / f"{grp['id']}_01_intensity_diffs_full.png"
fig1.save(fig1_path)
print("Сохранил:", fig1_path)

# ========= 2. Полноразмерное: бинаризация HR/LR/SR и различия =========
bin_hr, bin_lr, bin_sr, thr = binarize_from_hr_threshold(img_hr, img_lr, img_sr)
xor_hr_lr = xor_mask(bin_hr, bin_lr)
xor_hr_sr = xor_mask(bin_hr, bin_sr)
xor_sr_lr = xor_mask(bin_sr, bin_lr)

row1_b = hstack([
    make_tile2(bin_hr, f"HR bin (thr={thr})", upscale=1),
    make_tile2(bin_lr, "LR bin", upscale=1),
    make_tile2(xor_hr_lr, "HR⊕LR", upscale=1),
])
row2_b = hstack([
    make_tile2(bin_hr, f"HR bin (thr={thr})", upscale=1),
    make_tile2(bin_sr, "SR bin", upscale=1),
    make_tile2(xor_hr_sr, "HR⊕SR", upscale=1),
])
row3_b = hstack([
    make_tile2(xor_sr_lr, "SR⊕LR", upscale=1),
])

fig2 = vstack([row1_b, row2_b, row3_b])
fig2_path = OUT_DIR / f"{grp['id']}_02_binary_full.png"
fig2.save(fig2_path)
print("Сохранил:", fig2_path)

# ========= 3–4. То же самое для кастомного кропа =========
# Используем уже существующую crop_same_box
lr_crop, hr_crop, sr_crop, box, target_size = crop_same_box(img_lr, img_hr, img_sr)

diff_hr_lr_c = make_diff_img(hr_crop, lr_crop, amplify=4.0)
diff_hr_sr_c = make_diff_img(hr_crop, sr_crop, amplify=4.0)
diff_sr_lr_c = make_diff_img(sr_crop, lr_crop, amplify=4.0)

row1_c = hstack([
    make_tile2(hr_crop, "HR crop", upscale=UPSCALE),
    make_tile2(lr_crop, "LR crop", upscale=UPSCALE),
    make_tile2(diff_hr_lr_c, "|HR-LR| crop", upscale=UPSCALE),
])
row2_c = hstack([
    make_tile2(hr_crop, "HR crop", upscale=UPSCALE),
    make_tile2(sr_crop, "SR crop", upscale=UPSCALE),
    make_tile2(diff_hr_sr_c, "|HR-SR| crop", upscale=UPSCALE),
])
row3_c = hstack([
    make_tile2(diff_sr_lr_c, "|SR-LR| crop", upscale=UPSCALE),
])

fig3 = vstack([row1_c, row2_c, row3_c])
fig3_path = OUT_DIR / f"{grp['id']}_03_intensity_diffs_crop.png"
fig3.save(fig3_path)
print("Сохранил:", fig3_path)

bin_hr_c, bin_lr_c, bin_sr_c, thr_c = binarize_from_hr_threshold(hr_crop, lr_crop, sr_crop)
xor_hr_lr_c = xor_mask(bin_hr_c, bin_lr_c)
xor_hr_sr_c = xor_mask(bin_hr_c, bin_sr_c)
xor_sr_lr_c = xor_mask(bin_sr_c, bin_lr_c)

row1_bc = hstack([
    make_tile2(bin_hr_c, f"HR bin crop (thr={thr_c})", upscale=UPSCALE),
    make_tile2(bin_lr_c, "LR bin crop", upscale=UPSCALE),
    make_tile2(xor_hr_lr_c, "HR⊕LR crop", upscale=UPSCALE),
])
row2_bc = hstack([
    make_tile2(bin_hr_c, f"HR bin crop (thr={thr_c})", upscale=UPSCALE),
    make_tile2(bin_sr_c, "SR bin crop", upscale=UPSCALE),
    make_tile2(xor_hr_sr_c, "HR⊕SR crop", upscale=UPSCALE),
])
row3_bc = hstack([
    make_tile2(xor_sr_lr_c, "SR⊕LR crop", upscale=UPSCALE),
])

fig4 = vstack([row1_bc, row2_bc, row3_bc])
fig4_path = OUT_DIR / f"{grp['id']}_04_binary_crop.png"
fig4.save(fig4_path)
print("Сохранил:", fig4_path)

# ========= 5. Гистограммы интенсивностей HR/LR/SR (full + crop) =========
def plot_hists(hr, lr, sr, hr_c, lr_c, sr_c, out_path):
    hr_np = np.asarray(hr, dtype=np.uint8)
    lr_np = np.asarray(lr, dtype=np.uint8)
    sr_np = np.asarray(sr, dtype=np.uint8)

    hr_c_np = np.asarray(hr_c, dtype=np.uint8)
    lr_c_np = np.asarray(lr_c, dtype=np.uint8)
    sr_c_np = np.asarray(sr_c, dtype=np.uint8)

    fig, axes = plt.subplots(2, 3, figsize=(12, 6), sharex=True, sharey=True)
    bins = np.linspace(0, 255, 256)

    axes[0, 0].hist(hr_np.ravel(), bins=bins)
    axes[0, 0].set_title("HR full")
    axes[0, 1].hist(lr_np.ravel(), bins=bins)
    axes[0, 1].set_title("LR full")
    axes[0, 2].hist(sr_np.ravel(), bins=bins)
    axes[0, 2].set_title("SR full")

    axes[1, 0].hist(hr_c_np.ravel(), bins=bins)
    axes[1, 0].set_title("HR crop")
    axes[1, 1].hist(lr_c_np.ravel(), bins=bins)
    axes[1, 1].set_title("LR crop")
    axes[1, 2].hist(sr_c_np.ravel(), bins=bins)
    axes[1, 2].set_title("SR crop")

    for ax in axes.ravel():
        ax.set_xlabel("intensity")
        ax.set_ylabel("count")

    plt.tight_layout()
    fig.savefig(out_path, dpi=200)
    plt.close(fig)

fig5_path = OUT_DIR / f"{grp['id']}_05_histograms.png"
plot_hists(img_hr, img_lr, img_sr, hr_crop, lr_crop, sr_crop, fig5_path)
print("Сохранил:", fig5_path)

C:\Users\Вячеслав\AppData\Local\Temp\ipykernel_27284\274334411.py:43: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  diff = Image.fromarray(arr, mode="L")
C:\Users\Вячеслав\AppData\Local\Temp\ipykernel_27284\274334411.py:83: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(m, mode="L")
C:\Users\Вячеслав\AppData\Local\Temp\ipykernel_27284\274334411.py:92: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(xor_np, mode="L")


Сохранил: ..\..\sr_report_x2_msresunet\sample_0_01_intensity_diffs_full.png
Сохранил: ..\..\sr_report_x2_msresunet\sample_0_02_binary_full.png
Сохранил: ..\..\sr_report_x2_msresunet\sample_0_03_intensity_diffs_crop.png
Сохранил: ..\..\sr_report_x2_msresunet\sample_0_04_binary_crop.png
Сохранил: ..\..\sr_report_x2_msresunet\sample_0_05_histograms.png
